### **_Importar librerias_**

In [1]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [2]:
pd.set_option("display.max_columns",None)

### **_Lectura de datos_**

In [3]:

dbcp_ordenado= pd.read_excel("distr_barrios_codpostal.xlsx")
redpiso_normal= pd.read_excel("Redpiso.xlsx")
redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [4]:
# Bien aqui vamos a construir una funcion que recorra todo el dataframe y tome los caracteres especiales 
# las comas, los puntos, entre otros y los reemplaza con nada o sea basicamente eliminarlos. 

def limpiar_texto(x):
    return re.sub(r'[^\w\s]','',unidecode(str(x)))



In [5]:
def limpiar_valores(valor):
    if type(valor) == str:
        valor = re.sub(r'[^\w\s]','', valor)
        valor = valor.strip()
    return valor 

In [6]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

### **_Aqui se termina la union de los dos ficheros de pisos madrid y empezamos con redpiso_**

----------

In [7]:
## Aqui empieza el primer tratamiento de redpiso normal

In [8]:
redpiso_normal.columns

Index(['Título', 'Imagen', 'item', 'item1', 'propertylistoptionsitememblem',
       'propertylistoptionsitem', 'propertylistoptionsitem2',
       'propertylistoptionsitem3', 'itemlink_URL', 'itemlink', 'Imagen4',
       'Número', 'Número5', 'officelink_URL', 'officelink'],
      dtype='object')

In [9]:
# Extraer la última palabra después del guion 

redpiso_normal["referencia"] = redpiso_normal['itemlink_URL'].apply(lambda x: re.findall(r'[^-]+$', x)[0])

In [10]:
# Conversion de letras a minusculas en las columnas
redpiso_normal.columns= redpiso_normal.columns.str.lower()

In [11]:
# Columnas que seran eliminadas ya previamente analizadas.
columnas_a_eliminar = ["item","item1","imagen4","propertylistoptionsitem3","número5"]

redpiso_normal= redpiso_normal.drop(columns=columnas_a_eliminar)

In [12]:
# Renombramiento de la columna.
redpiso_normal = redpiso_normal.rename(columns={"título": "precio"})
redpiso_normal = redpiso_normal.rename(columns={"imagen": "url_imagen"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitememblem": "reservado"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem": "m2_constr"})
redpiso_normal = redpiso_normal.rename(columns={"propertylistoptionsitem2": "cantidad_habitaciones"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink_url": "url"})
redpiso_normal = redpiso_normal.rename(columns={"itemlink": "descripcion"})
redpiso_normal = redpiso_normal.rename(columns={"número": "contacto"})
redpiso_normal = redpiso_normal.rename(columns={"officelink": "nombre_oficina"})

In [13]:
# quitamos la palabra tel: y solo dejamos los valores numeros 
redpiso_normal["contacto"] = redpiso_normal["contacto"].str.replace(r'(?i)^tel:', "", regex=True)

In [14]:
# Cree una funcion que que toma los espacios y los caracteres especiales sin dejar algo detras.
def limpiar_valores(valor):
    if type(valor) == str:
        valor = re.sub(r'[^\w\s]','', valor)
        valor = valor.strip()
    return valor

# Cree una funcion corta que recorre cada valor del dataset y toma los valores que especificmaos en la funcion anterior.
#redpiso_normal= redpiso_normal.apply(lambda col: col.apply(limpiar_valores))

In [15]:
for col in redpiso_normal.columns: 
    if col != 'url': 
        redpiso_normal[col] = redpiso_normal[col].apply(limpiar_valores) 
     

In [16]:
# Rellena los valores nulos a sin_reserva
#redpiso_normal["reservado/sin_reserva"]= redpiso_normal["reservado/sin_reserva"].fillna("sin_reserva")

In [17]:
# Expresion regular que recorre la columna extrae los numeros de la cada valor y elimina lo demas.aplicada a la columna que se muestra.

redpiso_normal["cantidad_habitaciones"] = redpiso_normal["cantidad_habitaciones"].str.extract(r'(\d+)')

redpiso_normal["m2_constr"] = redpiso_normal["m2_constr"].str.extract(r'(\d+)')

In [18]:


# Asignar 1 a "Reservado", 0 a "Sin Reserva" y dejar el resto como están
redpiso_normal["reservado"] = redpiso_normal["reservado"].apply(lambda x: 1 if str(x).strip().upper() == "RESERVADO" else 0)




In [19]:
# Se convierte los valores RESERVADO a 1 y todo los demas a 0, basicamente los que aun no estan reservados seran 0.
#redpiso_normal["reservado/sin_reserva"] = redpiso_normal ["reservado/sin_reserva"].apply(lambda x : 1 if x.strip().upper()== "RESERVADO" else 0)

In [20]:
redpiso_normal.isna().sum()

precio                     0
url_imagen               723
reservado                  0
m2_constr                  7
cantidad_habitaciones      5
url                        0
descripcion                0
contacto                   0
officelink_url             0
nombre_oficina             0
referencia                 0
dtype: int64

In [21]:
redpiso_normal.shape

(2108, 11)

### **_Aqui se termina el tratamiento de redpiso_normal,ahora sigue el procesmiento de redpiso_detalles_**

In [22]:
redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [23]:
redpiso_detalles.head(3)

,Título,precio,Campo2,Campo3,Campo4,Campo5,Campo6,Tiempo,Tiempo1,Campo7,Campo8,Campo9,Campo10,Campo11,Campo12,Campo13,Tiempo2,Campo14,Campo15,Tiempo3,Campo16,Campo17,Campo18,Tiempo4,Campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500 €,"Piso en venta en CALLE BENIMAMET, San Cristóba...",ESTUPENDA VIVIENDA EN CALLE BENIMAMET,RP2842024125161,"Piso en venta en CALLE BENIMAMET, San Cristóba...",REF: RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,2.0,1.0,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000 €,"Piso en venta en Usera, Madrid, Madrid\n ...",ESTUDIO JARDINES DE YBARRA II,RP782024130863,"Piso en venta en Usera, Madrid, Madrid",REF: RP782024130863,Nº Visitas 714,02/10/2024,30 m²,1.0,1.0,ESTUDIO JARDINES DE YBARRA II,Redpiso vende en exclusiva ACOGEDOR ESTUDIO en...,Metros: 30 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,"Piso en venta en Buenavista, Carabanchel, Madr...",¡Tu nuevo hogar te espera en Buena Vista!,RP442024126007,"Piso en venta en Buenavista, Carabanchel, Madr...",REF: RP442024126007,Nº Visitas 3387,26/04/2024,37 m²,1.0,1.0,¡Tu nuevo hogar te espera en Buena Vista!,Redpiso vende en exclusiva este encantador est...,Metros: 37 m²,Metros útiles: 34 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 111 KW h/m² año,Emisiones: 18 Kg CO2/m² año,Estado: Perfecto Estado,Exterior,Orientación: Sur


In [24]:
# renombramiento de columnas 
redpiso_detalles = redpiso_detalles.rename( columns={"Campo1": "precio"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo2":"descripcion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo":"cantidad_visitas"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo7":"espacio_metros_cuadrados"})
redpiso_detalles = redpiso_detalles.rename(columns={"Tiempo1":"fecha_publicacion"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo6": "referencia"})
redpiso_detalles = redpiso_detalles.rename( columns={"título": "titulo"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo12":"m2_constr"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo13":"metros_utiles"})
redpiso_detalles = redpiso_detalles.rename(columns={"tiempo2": "cant_dormitorios"})
redpiso_detalles = redpiso_detalles.rename(columns={"Campo9": "banos"})
redpiso_detalles = redpiso_detalles.rename(columns={"campo15": "cant_armarios"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo14":"calidad_energetica"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo10":"titularanuncio"})
redpiso_detalles= redpiso_detalles.rename(columns={"campo11":"descripcion_detallada"})
#redpiso_detalles= redpiso_detalles.rename(columns={"campo4":"referencia"})

In [25]:
redpiso_detalles['Campo11'].value_counts(dropna=False)

Campo11
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [26]:
# eliminacion de columnas
redpiso_detalles= redpiso_detalles.drop("descripcion", axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo5", axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo9",axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo10",axis=1)
#redpiso_detalles = redpiso_detalles.drop ("Campo11", axis=1)
#redpiso_detalles = redpiso_detalles.drop("Campo12",axis=1)
redpiso_detalles = redpiso_detalles.drop("Campo8", axis= 1)
redpiso_detalles = redpiso_detalles.drop("Campo3", axis= 1)
redpiso_detalles = redpiso_detalles.drop("Campo4", axis= 1)

In [27]:
# Columnas a minusculas
redpiso_detalles.columns = redpiso_detalles.columns.str.lower()

In [28]:
redpiso_detalles.isna().sum()

título                        0
precio                        0
referencia                    0
cantidad_visitas              0
fecha_publicacion             0
espacio_metros_cuadrados      6
banos                        46
campo10                     728
campo11                      23
campo12                       2
campo13                       3
tiempo2                       4
campo14                      14
campo15                       8
tiempo3                       5
campo16                      20
campo17                      49
campo18                      82
tiempo4                     186
campo19                     123
dtype: int64

In [29]:
redpiso_detalles

,título,precio,referencia,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,banos,campo10,campo11,campo12,campo13,tiempo2,campo14,campo15,tiempo3,campo16,campo17,campo18,tiempo4,campo19
0,"Piso en venta en CALLE BENIMAMET, San Cristóba...",106.500 €,REF: RP2842024125161,Nº Visitas 5857,02/04/2024,64 m²,1.0,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 64 m²,Metros útiles: 60 m²,Dormitorios: 2,Exterior,Estado: Perfecto Estado,Baños: 1,Aire acondicionado,Agua caliente: Individual,Tipo de suelo: Tarima normal,Año de construcción: 1962,Antigüedad: Más de 50 años
1,"Piso en venta en Usera, Madrid, Madrid",119.000 €,REF: RP782024130863,Nº Visitas 714,02/10/2024,30 m²,1.0,ESTUDIO JARDINES DE YBARRA II,Redpiso vende en exclusiva ACOGEDOR ESTUDIO en...,Metros: 30 m²,Metros útiles: 27 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Estado: Perfecto Estado,Orientación: Sur,Exterior,Agua caliente: Individual,Cocina equipada
2,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,REF: RP442024126007,Nº Visitas 3387,26/04/2024,37 m²,1.0,¡Tu nuevo hogar te espera en Buena Vista!,Redpiso vende en exclusiva este encantador est...,Metros: 37 m²,Metros útiles: 34 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 111 KW h/m² año,Emisiones: 18 Kg CO2/m² año,Estado: Perfecto Estado,Exterior,Orientación: Sur
3,"Piso en venta en Buenavista, Carabanchel, Madr...",120.000 €,REF: RP442024126427,Nº Visitas 2924,10/05/2024,33 m²,1.0,¡Tu nuevo hogar te espera en Buena Vista!,Redpiso vende en exclusiva este encantador est...,Metros: 33 m²,Metros útiles: 30 m²,Dormitorios: 1,Cal. energética:,Armarios: 1,Baños: 1,Consumo: 145 KW h/m² año,Emisiones: 24 Kg CO2/m² año,Estado: Perfecto Estado,Agua caliente: Individual,Orientación: Oeste
4,"Piso en venta en CALLE BENIFERRI, San Cristóba...",127.500 €,REF: RP2842024130186,Nº Visitas 485,12/09/2024,58 m²,1.0,VIVIENDA EN CALLE BENIFERRI,"Redpiso San Cristobal de los Ángeles, inscrito...",Metros: 58 m²,Metros útiles: 50 m²,Dormitorios: 3,Exterior,Estado: Perfecto Estado,Baños: 1,Ascensor,Aire acondicionado,Agua caliente: Gas natural,Tipo de suelo: Terrazo,Calefacción: Gas Natural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,"Piso en venta en CALLE ORIO, 4, Los Ángeles, V...",A consultar,REF: RP332024126817,Nº Visitas 240,22/05/2024,NaN,NaN,NaN,NaN,Dormitorios: 3,Baños: 1,Cal. energética:,Exterior,Orientación: Norte,Estado: A reformar,Agua caliente: Gas natural,Calefacción: Gas Natural,Tipo de suelo: Baldosa,Antigüedad: Más de 50 años,Tipo de fachada: Cemento
2104,"Piso en venta en CALLE CANCION DEL OLVIDO, 12,...",A consultar,REF: RP332024127980,Nº Visitas 218,24/06/2024,90 m²,1.0,NaN,NaN,Metros: 90 m²,Metros útiles: 71 m²,Dormitorios: 3,Ascensor,Cal. energética:,Baños: 1,Agua caliente: Individual,Tipo de fachada: Otros materiales,Año de construcción: 1960,NaN,"Comunidad: 65,00 €"
2105,Piso en venta en CALLE LA ALEGRIA DE LA HUERTA...,A consultar,REF: RP332024128702,Nº Visitas 172,15/07/2024,79 m²,1.0,NaN,NaN,Metros: 79 m²,Metros útiles: 63 m²,Dormitorios: 3,Estado: Perfecto Estado,Cal. energética:,Baños: 1,Exterior,Ascensor,Aire acondicionado,Calefacción: Eléctrica,Agua caliente: Gas natural
2106,"Piso en venta en CALLE Ochagavia, Valdezarza, ...",A consultar,REF: RP402024127484,Nº Visitas 845,11/06/2024,76 m²,1.0,"Piso en venta en CALLE Ochagavia, Valdezarza, ...",Redpiso pone en venta un piso en calle Ochagav...,Metros: 76 m²,Metros útiles: 69 m²,Dormitorios: 3,Cal. energética:,Armarios: 2,Baños: 1,Consumo: 172 KW h/m² año,Emisiones: 36 Kg CO2/m² año,Orientación: Oeste,Ascensor,Exterior


In [30]:
# Eliminacion de las primeras 3 letras en la referencia web - REF- dejando solo la referencia 
#redpiso_detalles["referencia_web"] = redpiso_detalles["referencia_web"].str[3:].str.strip()

In [31]:
# Reemplace todas las letras ñ por n.
redpiso_detalles = redpiso_detalles.apply(lambda col : col.str.replace("ñ","n")if col.dtype == "object" else col)



#  Cree una funcion corta que recorre cada valor del dataset y toma los valores que especificmaos en la funcion anterior.
redpiso_detalles = redpiso_detalles.apply(lambda col: col.apply(limpiar_valores))


# Cree una funcion que elimina todos los acentos  y caracteres especiales
redpiso_detalles = redpiso_detalles.apply(lambda x: x.apply(lambda y: unidecode(y) if isinstance(y,str) else y )if x.dtype == "object" else x)



In [32]:
redpiso_detalles

,título,precio,referencia,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,banos,campo10,campo11,campo12,campo13,tiempo2,campo14,campo15,tiempo3,campo16,campo17,campo18,tiempo4,campo19
0,Piso en venta en CALLE BENIMAMET San Cristobal...,106500,REF RP2842024125161,No Visitas 5857,02042024,64 m2,1.0,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,Redpiso San Cristobal de los Angeles inscrito ...,Metros 64 m2,Metros utiles 60 m2,Dormitorios 2,Exterior,Estado Perfecto Estado,Banos 1,Aire acondicionado,Agua caliente Individual,Tipo de suelo Tarima normal,Ano de construccion 1962,Antiguedad Mas de 50 anos
1,Piso en venta en Usera Madrid Madrid,119000,REF RP782024130863,No Visitas 714,02102024,30 m2,1.0,ESTUDIO JARDINES DE YBARRA II,Redpiso vende en exclusiva ACOGEDOR ESTUDIO en...,Metros 30 m2,Metros utiles 27 m2,Dormitorios 1,Cal energetica,Armarios 1,Banos 1,Estado Perfecto Estado,Orientacion Sur,Exterior,Agua caliente Individual,Cocina equipada
2,Piso en venta en Buenavista Carabanchel Madrid...,120000,REF RP442024126007,No Visitas 3387,26042024,37 m2,1.0,Tu nuevo hogar te espera en Buena Vista,Redpiso vende en exclusiva este encantador est...,Metros 37 m2,Metros utiles 34 m2,Dormitorios 1,Cal energetica,Armarios 1,Banos 1,Consumo 111 KW hm2 ano,Emisiones 18 Kg CO2m2 ano,Estado Perfecto Estado,Exterior,Orientacion Sur
3,Piso en venta en Buenavista Carabanchel Madrid...,120000,REF RP442024126427,No Visitas 2924,10052024,33 m2,1.0,Tu nuevo hogar te espera en Buena Vista,Redpiso vende en exclusiva este encantador est...,Metros 33 m2,Metros utiles 30 m2,Dormitorios 1,Cal energetica,Armarios 1,Banos 1,Consumo 145 KW hm2 ano,Emisiones 24 Kg CO2m2 ano,Estado Perfecto Estado,Agua caliente Individual,Orientacion Oeste
4,Piso en venta en CALLE BENIFERRI San Cristobal...,127500,REF RP2842024130186,No Visitas 485,12092024,58 m2,1.0,VIVIENDA EN CALLE BENIFERRI,Redpiso San Cristobal de los Angeles inscrito ...,Metros 58 m2,Metros utiles 50 m2,Dormitorios 3,Exterior,Estado Perfecto Estado,Banos 1,Ascensor,Aire acondicionado,Agua caliente Gas natural,Tipo de suelo Terrazo,Calefaccion Gas Natural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,Piso en venta en CALLE ORIO 4 Los Angeles Vill...,A consultar,REF RP332024126817,No Visitas 240,22052024,NaN,NaN,NaN,NaN,Dormitorios 3,Banos 1,Cal energetica,Exterior,Orientacion Norte,Estado A reformar,Agua caliente Gas natural,Calefaccion Gas Natural,Tipo de suelo Baldosa,Antiguedad Mas de 50 anos,Tipo de fachada Cemento
2104,Piso en venta en CALLE CANCION DEL OLVIDO 12 L...,A consultar,REF RP332024127980,No Visitas 218,24062024,90 m2,1.0,NaN,NaN,Metros 90 m2,Metros utiles 71 m2,Dormitorios 3,Ascensor,Cal energetica,Banos 1,Agua caliente Individual,Tipo de fachada Otros materiales,Ano de construccion 1960,NaN,Comunidad 6500
2105,Piso en venta en CALLE LA ALEGRIA DE LA HUERTA...,A consultar,REF RP332024128702,No Visitas 172,15072024,79 m2,1.0,NaN,NaN,Metros 79 m2,Metros utiles 63 m2,Dormitorios 3,Estado Perfecto Estado,Cal energetica,Banos 1,Exterior,Ascensor,Aire acondicionado,Calefaccion Electrica,Agua caliente Gas natural
2106,Piso en venta en CALLE Ochagavia Valdezarza Mo...,A consultar,REF RP402024127484,No Visitas 845,11062024,76 m2,1.0,Piso en venta en CALLE Ochagavia Valdezarza Mo...,Redpiso pone en venta un piso en calle Ochagav...,Metros 76 m2,Metros utiles 69 m2,Dormitorios 3,Cal energetica,Armarios 2,Banos 1,Consumo 172 KW hm2 ano,Emisiones 36 Kg CO2m2 ano,Orientacion Oeste,Ascensor,Exterior


In [33]:
valores_buscar1= ["Dormitorios 1", "Dormitorios 2", "Dormitorios 3", "Dormitorios 4","Dormitorios1", "Dormitorios2","Dormitorios3", "Dormitorios4",
                  "Dormitorios 5", "Dormitorios 6", "Dormitorios 7", "Dormitorios 8","Dormitorios  8", "Dormitorios 9", "Dormitorios 10", "Dormitorios5","Dormitorios6",
                  "Dormitorios7", "Dormitorios8", "Dormitoros9"]

valores_buscar2= ["Banos 1", "Banos 2", "Banos 3", "Banos 4","Banos 5","Banos 6","Banos 7","Banos 8",
                   "Banos1", "Banos2", "Banos3", "Banos4", "Banos5", "Banos6", "Banos7","Banos8"]

nueva_columna = "dormitorios"
nueva_columna2 = "banos"



In [34]:
# Creacion de columnas nuevas y rellenarlos con nulos.  Despues iterar sobre las columnas y rellenarlas con los valores declararlos en las listas de atras


redpiso_detalles[nueva_columna]= np.nan
redpiso_detalles[nueva_columna2]= np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles .columns[:-1]:
        if pd.notna(row[col]):
            for valor in valores_buscar1:
                if valor in str(row[col]):
                    redpiso_detalles.at[index, nueva_columna] =valor 
                    redpiso_detalles.at[index, col]= np.nan
                    break




for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles .columns[:-1]:
        if pd.notna(row[col]):
            for valor in valores_buscar2:
                if valor in str(row[col]):
                    redpiso_detalles.at[index, nueva_columna2] =valor 
                    redpiso_detalles.at[index, col]= np.nan
                    break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\3484431668.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Dormitorios 2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna] =valor
C:\Users\extas\AppData\Local\Temp\ipykernel_20344\3484431668.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Banos 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna2] =valor


In [35]:

nueva_columna3= "cantidad_armarios"
patron = r'(?i)^armario' 
redpiso_detalles[nueva_columna3]= np.nan

redpiso_detalles[nueva_columna3] = redpiso_detalles[nueva_columna3].astype("object") # Conversion de columna float64 a object.

# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron, regex=True).any():
                redpiso_detalles.at[index, nueva_columna3]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [36]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna4 = "consumo_energia"
patron_2 = r'(?i)^consumo'
redpiso_detalles[nueva_columna4]= np.nan
redpiso_detalles[nueva_columna4] = redpiso_detalles[nueva_columna4].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_2, regex=True).any():
                redpiso_detalles.at[index, nueva_columna4]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [37]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna5 = "emisiones_CO2"
patron_3 = r'(?i)^emisiones'
redpiso_detalles[nueva_columna5]= np.nan
redpiso_detalles[nueva_columna5] = redpiso_detalles[nueva_columna5].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_3, regex=True).any():
                redpiso_detalles.at[index, nueva_columna5]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [38]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna6 = "cal_energetica"
patron_4 = r'(?i)^cal'
redpiso_detalles[nueva_columna6]= np.nan
redpiso_detalles[nueva_columna6] = redpiso_detalles[nueva_columna6].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_4, regex=True).any():
                redpiso_detalles.at[index, nueva_columna6]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break



In [39]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna7="estado"
patron_5 =  r'(?i)^estado'
redpiso_detalles[nueva_columna7]= np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_5, regex=True).any():
                redpiso_detalles.at[index, nueva_columna7]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1235483781.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Estado Perfecto Estado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna7]= row[col]


In [40]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna8 = "antiguedad"
patron_6= r'(?i)^antiguedad'
redpiso_detalles[nueva_columna8]= np.nan
redpiso_detalles[nueva_columna8] = redpiso_detalles[nueva_columna8].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_6, regex=True).any():
                redpiso_detalles.at[index, nueva_columna8]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [41]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna9 = "ascensor"
patron_7= r'(?i)^ascensor'
redpiso_detalles[nueva_columna9]= np.nan
redpiso_detalles[nueva_columna9] = redpiso_detalles[nueva_columna9].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_7, regex=True).any():
                redpiso_detalles.at[index, nueva_columna9]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [42]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna10 = "aire_acondicionado"
patron_8= r'(?i)^aire'
redpiso_detalles[nueva_columna10]= np.nan
redpiso_detalles[nueva_columna10] = redpiso_detalles[nueva_columna10].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_8, regex=True).any():
                redpiso_detalles.at[index, nueva_columna10]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [43]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna11 = "orientacion"
patron_9= r'(?i)^orientacion'
redpiso_detalles[nueva_columna11]= np.nan
redpiso_detalles[nueva_columna11] = redpiso_detalles[nueva_columna11].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_9, regex=True).any():
                redpiso_detalles.at[index, nueva_columna11]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [44]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna12 = "agua_caliente"
patron_10= r'(?i)^agua'
redpiso_detalles[nueva_columna12]= np.nan
redpiso_detalles[nueva_columna12] = redpiso_detalles[nueva_columna12].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_10, regex=True).any():
                redpiso_detalles.at[index, nueva_columna12]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [45]:
patron_11_antiguedad= r'^ano|año'

# Aqui a diferencia de los otros bucles iterativos se busco integrar los valores que iniciaran en ano y año a una columna ya existente.
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_11_antiguedad, regex=True, flags=re.IGNORECASE).any():
                redpiso_detalles.at[index,nueva_columna8]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [46]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna13 = "calefaccion"
patron_12= r'(?i)^calefaccion'
redpiso_detalles[nueva_columna13]= np.nan
redpiso_detalles[nueva_columna13] = redpiso_detalles[nueva_columna13].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_12, regex=True).any():
                redpiso_detalles.at[index, nueva_columna13]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [47]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna14 = "Tipo de suelo_fachada"
patron_13= r'(?i)^tipo'
redpiso_detalles[nueva_columna14]= np.nan
redpiso_detalles[nueva_columna14] = redpiso_detalles[nueva_columna14].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_13, regex=True).any():
                redpiso_detalles.at[index, nueva_columna14]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break


# patron_14_tipo= r'^\S*tipo' #  Asegura que no haya espacios antes de palabra. Expresion regular.
# # Aqui se ejerce la busqueda de tipo pero reconociendo espacios antes de la palabra. Despues de coloca en la columna tipo de suelo_fachada.
# for index, row in datos.iterrows():
#     for col in datos.columns[:-1]:
#         if pd.notna(row[col]):
#             valor= str(row[col])
#             if pd.Series(valor).str.contains(patron_14_tipo, regex=True, flags=re.IGNORECASE).any():
#                 datos.at[index,nueva_columna14]= row[col]
#                 datos.at[index, col]= np.nan
#                 break

In [48]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna15 = "cocina_equipada"
patron_15= r'(?i)^cocina'


redpiso_detalles[nueva_columna15]= np.nan
redpiso_detalles[nueva_columna15] = redpiso_detalles[nueva_columna15].astype("object")
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_15, regex=True).any():
                redpiso_detalles.at[index, nueva_columna15]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [49]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

nueva_columna16 = "garaje"
patron_16= r'(?i)^garajes'
redpiso_detalles[nueva_columna16]= np.nan
redpiso_detalles[nueva_columna16] = redpiso_detalles[nueva_columna16].astype("object")

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_16, regex=True).any():
                redpiso_detalles.at[index, nueva_columna16]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break



In [50]:
# Aqui se recorre el dataset para tomar los valores indicados en en el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_17_metros= r'(?i)^metros'
nueva_columna17 = "m2_utiles"
redpiso_detalles[nueva_columna17] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_17_metros, regex=True).any():
                redpiso_detalles.at[index, nueva_columna17]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

# patron_18_metros= r'^\S*metros'
# for index, row in redpiso_detalles.iterrows():
#     for col in redpiso_detalles.columns[:-1]:
#         if pd.notna(row[col]):
#             valor= str(row[col])
#             if pd.Series(valor).str.contains(patron_18_metros, regex=True, flags=re.IGNORECASE).any():
#                 redpiso_detalles.at[index,nueva_columna17]= row[col]
#                 redpiso_detalles.at[index, col]= np.nan
#                 break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\3118533722.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Metros 64 m2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna17]= row[col]


In [51]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_20= r'(?i)^exterior'
nueva_columna18 = "vista_zona"
redpiso_detalles[nueva_columna18] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_20, regex=True).any():
                redpiso_detalles.at[index, nueva_columna18]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break


C:\Users\extas\AppData\Local\Temp\ipykernel_20344\2806881563.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Exterior' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna18]= row[col]


In [52]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_21= r'(?i)^piscina'
nueva_columna19 = "piscina"
redpiso_detalles[nueva_columna19] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_21, regex=True).any():
                redpiso_detalles.at[index, nueva_columna19]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\995817297.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Piscina' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna19]= row[col]


In [53]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_22= r'(?i)^trastero'
nueva_columna22 = "trastero"
redpiso_detalles[nueva_columna22] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_22, regex=True).any():
                redpiso_detalles.at[index, nueva_columna22]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\4053109142.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trastero' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna22]= row[col]


In [54]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_23= r'(?i)^comunidad'
nueva_columna23 = "gastos_comunidad"
redpiso_detalles[nueva_columna23] = np.nan


for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_23, regex=True).any():
                redpiso_detalles.at[index, nueva_columna23]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\2512737850.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Comunidad 7400' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna23]= row[col]


In [55]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_24= r'(?i)^zonas'
nueva_columna24 = "Zonas_verdes"
redpiso_detalles[nueva_columna24] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_24, regex=True).any():
                redpiso_detalles.at[index, nueva_columna24]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\2808801515.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Zonas verdes' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna24]= row[col]


In [56]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_25= r'(?i)^numero'
nueva_columna25 = "terrazas"
redpiso_detalles[nueva_columna25] = np.nan


for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_25, regex=True).any():
                redpiso_detalles.at[index, nueva_columna25]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1566383252.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Numero de terrazas 1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna25]= row[col]


In [57]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_26= r'(?i)^acceso'
nueva_columna26 = "acceso_minusvalido"
redpiso_detalles[nueva_columna26] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_26, regex=True).any():
                redpiso_detalles.at[index, nueva_columna26]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\3499825224.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Acceso minusvalidos' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna26]= row[col]


In [58]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_27= r'(?i)^puerta'
nueva_columna27 = "puerta_seguridad"
redpiso_detalles[nueva_columna27] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_27, regex=True).any():
                redpiso_detalles.at[index, nueva_columna27]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1113838515.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Puerta de seguridad' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna27]= row[col]


In [59]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_28= r'(?i)^conserje'
nueva_columna28 = "conserje"
redpiso_detalles[nueva_columna28] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_28, regex=True).any():
                redpiso_detalles.at[index, nueva_columna28]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\2160241313.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Conserje' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna28]= row[col]


In [60]:
# Aqui se filtra la palabra Conserje pero sin distincion de espacios antes de la palabra. 
patron_29_conserje= r'^\S*Conserje'
for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_29_conserje, regex=True, flags=re.IGNORECASE).any():
                redpiso_detalles.at[index,nueva_columna28]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

In [61]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_30= r'(?i)^plantas'

nueva_columna29 = "num_plantas"

redpiso_detalles[nueva_columna29] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_30, regex=True).any():
                redpiso_detalles.at[index, nueva_columna29]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\757634582.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Plantas 5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna29]= row[col]


In [62]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_31= r'(?i)^pisos'

nueva_columna30 = "num_pisos"

redpiso_detalles[nueva_columna30] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_31, regex=True).any():
                redpiso_detalles.at[index, nueva_columna30]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1093162397.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Pisos 25' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna30]= row[col]


In [63]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_32= r'(?i)^alarma'

nueva_columna31 = "alarma"

redpiso_detalles[nueva_columna31] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_32, regex=True).any():
                redpiso_detalles.at[index, nueva_columna31]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1007444481.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Alarma' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna31]= row[col]


In [64]:
# Aqui se recorre el dataset para tomar los valores que tengan el patron que se dio y colocarlos en la nueva columna 
# dejando nulos en los lugares originales.

patron_33= r'(?i)^amueblado'

nueva_columna32 = "amueblado"

redpiso_detalles[nueva_columna32] = np.nan

for index, row in redpiso_detalles.iterrows():
    for col in redpiso_detalles.columns[:-1]:
        if pd.notna(row[col]):
            valor= str(row[col])
            if pd.Series(valor).str.contains(patron_33, regex=True).any():
                redpiso_detalles.at[index, nueva_columna32]= row[col]
                redpiso_detalles.at[index, col]= np.nan
                break

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1403479313.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Amueblado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  redpiso_detalles.at[index, nueva_columna32]= row[col]


In [65]:
redpiso_detalles["campo19"].value_counts()

campo19
Tipo de fachada Ladrillo            44
Tipo de suelo Parquet               31
Tipo de suelo Gres                  27
Tipo de suelo Tarima normal         15
Tipo de fachada Cemento             13
Tipo de fachada Mono Capa           11
Tipo de suelo Tarima Flotante       10
Tipo de suelo Baldosa                6
Tipo de suelo Otros                  5
Tipo de suelo Terrazo                3
Tipo de suelo Porcelanico            2
Tipo de suelo Sintasol               2
Tipo de fachada Piedra               2
Tipo de suelo Mosaico Hidraulico     1
Name: count, dtype: int64

In [66]:
# aqui intento extraer los valores los valores numericos que hay cuando se encuentra la palabra metros en la columna tiempo2 y las llevo a m_cuadrados_utiles

def mover_valores_metros(df):
   
    # Expresión regular para buscar "metros" seguido de un número
    patron = r'(?i)\s*metros?.*?(\d+)\s*m2'  # "metros" o "metros" seguido de un número

    # Iterar sobre las filas del DataFrame
    for idx, row in df.iterrows():
        tiempo2_valor = str(row['tiempo2']).strip()  # Asegurarse de eliminar los espacios antes y después
        
        # Depuración: Mostrar valor de tiempo2
        print(f"Fila {idx}: valor en 'tiempo2': {tiempo2_valor}")
        
        # Buscar si el valor en la columna 'tiempo2' contiene 'metros' seguido de un número
        match = re.search(patron, tiempo2_valor)
        
        # Depuración: Mostrar si se encontró una coincidencia
        if match:
            metros_valor = match.group(1)  # Extraer el número que sigue a "metros"
            print(f"Se encontró 'metros' en la fila {idx}: {metros_valor}")  # Depuración
            # Si la columna 'm_cuadrados_utiles' está vacía, mover el valor de 'tiempo2'
            if pd.isna(df.at[idx, 'm2_utiles']):
                df.at[idx, 'm2_utiles'] = metros_valor  # Transferir el número a 'm2_utiles
                df.at[idx, 'tiempo2'] = np.nan  # Reemplazar el valor en 'tiempo2' por NaN
                print(f"Fila {idx}: Se movió el valor a 'm2_utiles y se vació 'tiempo2'.")  # Depuración
            else:
                print(f"Fila {idx}: 'm2_utiles ya tiene valor, no se hizo el cambio.")  # Depuración

    return df

# Llamar a la función sobre tu DataFrame
redpiso_detalles2 = mover_valores_metros(redpiso_detalles)

# Ver los cambios en las columnas relevantes
print(redpiso_detalles2[['tiempo2', 'm2_utiles']].head())





# Hay datos que se filtraron correctamente a las columnas correspondientes como en m_cuadrados_utiles pero que no se retiraron correctamente 
# de otras columnas donde se tomaron los valores. Asi que habra valores en otras columnas pero ya estaran en la columna correspondiente tambien

Fila 0: valor en 'tiempo2': nan
Fila 1: valor en 'tiempo2': nan
Fila 2: valor en 'tiempo2': nan
Fila 3: valor en 'tiempo2': nan
Fila 4: valor en 'tiempo2': nan
Fila 5: valor en 'tiempo2': nan
Fila 6: valor en 'tiempo2': nan
Fila 7: valor en 'tiempo2': nan
Fila 8: valor en 'tiempo2': nan
Fila 9: valor en 'tiempo2': nan
Fila 10: valor en 'tiempo2': nan
Fila 11: valor en 'tiempo2': nan
Fila 12: valor en 'tiempo2': nan
Fila 13: valor en 'tiempo2': nan
Fila 14: valor en 'tiempo2': nan
Fila 15: valor en 'tiempo2': nan
Fila 16: valor en 'tiempo2': nan
Fila 17: valor en 'tiempo2': nan
Fila 18: valor en 'tiempo2': nan
Fila 19: valor en 'tiempo2': nan
Fila 20: valor en 'tiempo2': nan
Fila 21: valor en 'tiempo2': nan
Fila 22: valor en 'tiempo2': nan
Fila 23: valor en 'tiempo2': nan
Fila 24: valor en 'tiempo2': nan
Fila 25: valor en 'tiempo2': nan
Fila 26: valor en 'tiempo2': nan
Fila 27: valor en 'tiempo2': nan
Fila 28: valor en 'tiempo2': nan
Fila 29: valor en 'tiempo2': nan
Fila 30: valor en 't

In [67]:
# eliminacion de columnas 
redpiso_detalles= redpiso_detalles.drop(columns=["campo13","tiempo2","campo14","campo15","tiempo3","campo16","campo17","campo18","tiempo4",
"campo19"])

In [68]:
redpiso_detalles.shape

(2108, 39)

In [69]:
# Reemplazo de valores nulos por ceros y los otros por unos 
columnas_binario= ["amueblado","alarma","conserje","puerta_seguridad","acceso_minusvalido","cocina_equipada","aire_acondicionado",
                   "ascensor","piscina","trastero","Zonas_verdes"]

redpiso_detalles[columnas_binario]= redpiso_detalles[columnas_binario].notnull().astype(int)


In [70]:
redpiso_detalles.head()

,título,precio,referencia,cantidad_visitas,fecha_publicacion,espacio_metros_cuadrados,banos,campo10,campo11,campo12,dormitorios,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,estado,antiguedad,ascensor,aire_acondicionado,orientacion,agua_caliente,calefaccion,Tipo de suelo_fachada,cocina_equipada,garaje,m2_utiles,vista_zona,piscina,trastero,gastos_comunidad,Zonas_verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,num_plantas,num_pisos,alarma,amueblado
0,Piso en venta en CALLE BENIMAMET San Cristobal...,106500,REF RP2842024125161,No Visitas 5857,02042024,64 m2,Banos 1,ESTUPENDA VIVIENDA EN CALLE BENIMAMET,Redpiso San Cristobal de los Angeles inscrito ...,NaN,Dormitorios 2,NaN,NaN,NaN,NaN,Estado Perfecto Estado,Ano de construccion 1962,0,1,NaN,Agua caliente Individual,NaN,Tipo de suelo Tarima normal,0,NaN,Metros 64 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
1,Piso en venta en Usera Madrid Madrid,119000,REF RP782024130863,No Visitas 714,02102024,30 m2,Banos 1,ESTUDIO JARDINES DE YBARRA II,Redpiso vende en exclusiva ACOGEDOR ESTUDIO en...,NaN,Dormitorios 1,Armarios 1,NaN,NaN,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,Agua caliente Individual,NaN,NaN,1,NaN,Metros 30 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
2,Piso en venta en Buenavista Carabanchel Madrid...,120000,REF RP442024126007,No Visitas 3387,26042024,37 m2,Banos 1,Tu nuevo hogar te espera en Buena Vista,Redpiso vende en exclusiva este encantador est...,NaN,Dormitorios 1,Armarios 1,Consumo 111 KW hm2 ano,Emisiones 18 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Sur,NaN,NaN,NaN,0,NaN,Metros 37 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
3,Piso en venta en Buenavista Carabanchel Madrid...,120000,REF RP442024126427,No Visitas 2924,10052024,33 m2,Banos 1,Tu nuevo hogar te espera en Buena Vista,Redpiso vende en exclusiva este encantador est...,NaN,Dormitorios 1,Armarios 1,Consumo 145 KW hm2 ano,Emisiones 24 Kg CO2m2 ano,Cal energetica,Estado Perfecto Estado,NaN,0,0,Orientacion Oeste,Agua caliente Individual,NaN,NaN,0,NaN,Metros 33 m2,NaN,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0
4,Piso en venta en CALLE BENIFERRI San Cristobal...,127500,REF RP2842024130186,No Visitas 485,12092024,58 m2,Banos 1,VIVIENDA EN CALLE BENIFERRI,Redpiso San Cristobal de los Angeles inscrito ...,NaN,Dormitorios 3,NaN,NaN,NaN,NaN,Estado Perfecto Estado,NaN,1,1,NaN,Agua caliente Gas natural,Calefaccion Gas Natural,Tipo de suelo Terrazo,0,NaN,Metros 58 m2,Exterior,0,0,NaN,0,NaN,0,0,0,NaN,NaN,0,0


-----------

In [71]:
redpiso_detalles2= redpiso_detalles
redpiso_normal2= redpiso_normal

In [72]:
redpiso_detalles2.columns

Index(['título', 'precio', 'referencia', 'cantidad_visitas',
       'fecha_publicacion', 'espacio_metros_cuadrados', 'banos', 'campo10',
       'campo11', 'campo12', 'dormitorios', 'cantidad_armarios',
       'consumo_energia', 'emisiones_CO2', 'cal_energetica', 'estado',
       'antiguedad', 'ascensor', 'aire_acondicionado', 'orientacion',
       'agua_caliente', 'calefaccion', 'Tipo de suelo_fachada',
       'cocina_equipada', 'garaje', 'm2_utiles', 'vista_zona', 'piscina',
       'trastero', 'gastos_comunidad', 'Zonas_verdes', 'terrazas',
       'acceso_minusvalido', 'puerta_seguridad', 'conserje', 'num_plantas',
       'num_pisos', 'alarma', 'amueblado'],
      dtype='object')

In [73]:
redpiso_normal2.columns      #A diferencia del notebook donde se hizo la el trabajo inicialmente aqui tengo la columna officeling_url 

Index(['precio', 'url_imagen', 'reservado', 'm2_constr',
       'cantidad_habitaciones', 'url', 'descripcion', 'contacto',
       'officelink_url', 'nombre_oficina', 'referencia'],
      dtype='object')

In [74]:
redpiso_normal2.isna().sum()

precio                     0
url_imagen               723
reservado                  0
m2_constr                  7
cantidad_habitaciones      5
url                        0
descripcion                0
contacto                   0
officelink_url             0
nombre_oficina             0
referencia                 0
dtype: int64

In [75]:
redpiso_detalles2.shape

(2108, 39)

In [76]:
redpiso_normal2.shape

(2108, 11)

In [77]:
#redpiso_detalles2["dormitorios"] = redpiso_detalles2["dormitorios"].fillna(redpiso_normal2["habitaciones"].reindex(redpiso_normal2.index))

In [78]:
redpiso_detalles2["precio"] = redpiso_detalles2["precio"] .replace("A consultar",np.nan)
redpiso_normal2["precio"] = redpiso_normal2["precio"] .replace("A consultar",np.nan)

In [79]:
redpiso_detalles2["dormitorios"]= redpiso_detalles2["dormitorios"].astype(str)
redpiso_detalles2["banos"]= redpiso_detalles2["banos"].astype(str)
redpiso_detalles2["espacio_metros_cuadrados"]= redpiso_detalles2["espacio_metros_cuadrados"].astype(str) 
redpiso_detalles2["cantidad_armarios"]= redpiso_detalles2["cantidad_armarios"].astype(str)
redpiso_detalles2["cantidad_visitas"]= redpiso_detalles2["cantidad_visitas"].astype(str)

In [80]:
# Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bdormitorios\b', '', palabra)

redpiso_detalles2["dormitorios"]= redpiso_detalles2["dormitorios"].apply(eliminar_palabra)

# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbano\b', '', palabra)

redpiso_detalles2["banos"]= redpiso_detalles2["banos"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bbanos\b', '', palabra)

redpiso_detalles2["banos"]= redpiso_detalles2["banos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

redpiso_detalles2["espacio_metros_cuadrados"]= redpiso_detalles2["espacio_metros_cuadrados"].apply(eliminar_palabra)

# -----------------------------------------------------------------------------
def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bArmarios\b', '', str(palabra))

redpiso_detalles2["cantidad_armarios"] = redpiso_detalles2["cantidad_armarios"].apply(eliminar_palabra)

# -------------------------------------------------------------------------------

def eliminar_no_visitas(texto):
    # Eliminar "No Visitas" y dejar solo el número que le sigue
    return re.sub(r'(?i)No Visitas\s*(\d+)', r'\1', str(texto))

# Aplicar la función a la columna "cantidad_visitas"
redpiso_detalles2["cantidad_visitas"] = redpiso_detalles2["cantidad_visitas"].apply(eliminar_no_visitas)

In [81]:
redpiso_detalles2["m2_utiles"] = redpiso_detalles2["m2_utiles"].str.replace("Metros utiles", "", case=False, regex= True)
redpiso_detalles2["m2_utiles"] = redpiso_detalles2["m2_utiles"].str.replace("m2", "", case=False, regex= True)

In [82]:
# datos = pd.read_csv("avanzadonuevo_datos_redpiso_detalles.csv") en estos reemplzamientos se encontraba basicmanete el redpisodetalle_tercertratamiento

# datos = pd.read_csv("avanzadonuevo_datos_redpiso_detalles.csv")

In [83]:
# redpiso_detalles['fecha_publicacion'] = pd.to_datetime(redpiso_detalles['fecha_publicacion'].astype(str), format='%d%m%Y')

In [84]:
col_tiponuevo= ["precio","espacio_metros_cuadrados",
                "dormitorios","banos",
                "cantidad_armarios"]

redpiso_detalles2 = redpiso_detalles2.replace("nan",0)
redpiso_detalles2[col_tiponuevo]=redpiso_detalles2[col_tiponuevo].fillna(0)

redpiso_detalles2[col_tiponuevo] = redpiso_detalles2[col_tiponuevo].astype("float")

redpiso_detalles2[col_tiponuevo]=redpiso_detalles2[col_tiponuevo].replace(0,np.nan)


In [85]:
redpiso_normal2["precio"].isna().sum()

640

In [86]:
redpiso_detalles2=redpiso_detalles2.dropna(subset=["precio"])
redpiso_normal2= redpiso_normal2.dropna(subset=["precio"])

In [87]:
redpiso_detalles2.shape

(1468, 39)

In [88]:
redpiso_normal2.shape

(1468, 11)

In [89]:
nuevos_nombres={
    "descripcion":"titulo",
    "cantidad_habitaciones":"cantidad_dormitorios"
}
redpiso_normal2= redpiso_normal2.rename(columns=nuevos_nombres)

In [90]:
#redpiso_detalles2["espacio_metros_cuadrados"]=redpiso_detalles2["espacio_metros_cuadrados"].fillna(0) #Tenia dos nulos 
#redpiso_detalles2["dormitorios"]=redpiso_detalles2["cantidad_dormitorios"].fillna(0) #Tenia dos nulos

In [91]:
redpiso_detalles2["precio"]=redpiso_detalles2["precio"].astype("float")
redpiso_detalles2["dormitorios"]=redpiso_detalles2["dormitorios"].astype("float")
redpiso_detalles2["espacio_metros_cuadrados"]=redpiso_detalles2["espacio_metros_cuadrados"].astype("float")

In [92]:
redpiso_normal2.head(5)

,precio,url_imagen,reservado,m2_constr,cantidad_dormitorios,url,titulo,contacto,officelink_url,nombre_oficina,referencia
0,106500,NaN,1,64,2,https://www.redpiso.es/inmueble/piso-en-venta-...,Piso en venta en CALLE BENIMAMET San Cristóbal...,919208536,httpswwwredpisoesoficinainmobiliariasancristob...,REDPISO SAN CRISTOBAL,RP2842024125161
1,119000,NaN,0,30,1,https://www.redpiso.es/inmueble/piso-en-venta-...,Piso en venta en Usera Madrid Madrid,914282010,httpswwwredpisoesoficinainmobiliariauseraorcas...,REDPISO USERAORCASITAS,RP782024130863
2,120000,NaN,1,37,1,https://www.redpiso.es/inmueble/piso-en-venta-...,Piso en venta en Buenavista Carabanchel Madrid...,915996359,httpswwwredpisoesoficinainmobiliariapaucaraban...,REDPISO PAU CARABANCHEL,RP442024126007
3,120000,NaN,1,33,1,https://www.redpiso.es/inmueble/piso-en-venta-...,Piso en venta en Buenavista Carabanchel Madrid...,915996359,httpswwwredpisoesoficinainmobiliariapaucaraban...,REDPISO PAU CARABANCHEL,RP442024126427
4,127500,httpswwwredpisoesuploadpropertiesthumbsimg_951...,1,58,3,https://www.redpiso.es/inmueble/piso-en-venta-...,Piso en venta en CALLE BENIFERRI San Cristóbal...,919208536,httpswwwredpisoesoficinainmobiliariasancristob...,REDPISO SAN CRISTOBAL,RP2842024130186


In [93]:
redpiso_normal2.dtypes

precio                  object
url_imagen              object
reservado                int64
m2_constr               object
cantidad_dormitorios    object
url                     object
titulo                  object
contacto                object
officelink_url          object
nombre_oficina          object
referencia              object
dtype: object

In [94]:
#redpiso_normal2["espacio_metros_cuadrados"]=redpiso_normal2["espacio_metros_cuadrados"].fillna(0) #Tenia 4 nulos 
#redpiso_normal2["cantidad_dormitorios"]=redpiso_normal2["cantidad_dormitorios"].fillna(0) #Tenia 3 nulos

In [95]:
redpiso_normal2["precio"]=redpiso_normal2["precio"].astype("float")
#redpiso_normal2["espacio_metros_cuadrados"]=redpiso_normal2["espacio_metros_cuadrados"].astype("float")



In [96]:
redpiso_detalles2.dtypes

título                       object
precio                      float64
referencia                   object
cantidad_visitas             object
fecha_publicacion            object
espacio_metros_cuadrados    float64
banos                       float64
campo10                      object
campo11                      object
campo12                      object
dormitorios                 float64
cantidad_armarios           float64
consumo_energia              object
emisiones_CO2                object
cal_energetica               object
estado                       object
antiguedad                   object
ascensor                      int32
aire_acondicionado            int32
orientacion                  object
agua_caliente                object
calefaccion                  object
Tipo de suelo_fachada        object
cocina_equipada               int32
garaje                       object
m2_utiles                    object
vista_zona                   object
piscina                     

In [97]:
redpiso_detalles2 = redpiso_detalles2.rename(columns={"título":"titulo"})

In [98]:
redpiso_normal2.columns

Index(['precio', 'url_imagen', 'reservado', 'm2_constr',
       'cantidad_dormitorios', 'url', 'titulo', 'contacto', 'officelink_url',
       'nombre_oficina', 'referencia'],
      dtype='object')

In [99]:
redpiso_detalles2.shape,redpiso_normal2.shape

((1468, 39), (1468, 11))

In [100]:
redpiso_detalles2['titulo'].equals(redpiso_detalles2['titulo']) 

True

In [101]:
#elimino columnas iguales de los 2 dataframes
colseliminar=["titulo",'precio']
redpiso_detalles2.drop(columns=colseliminar,inplace=True)

In [102]:
redpiso_detalles2.columns

Index(['referencia', 'cantidad_visitas', 'fecha_publicacion',
       'espacio_metros_cuadrados', 'banos', 'campo10', 'campo11', 'campo12',
       'dormitorios', 'cantidad_armarios', 'consumo_energia', 'emisiones_CO2',
       'cal_energetica', 'estado', 'antiguedad', 'ascensor',
       'aire_acondicionado', 'orientacion', 'agua_caliente', 'calefaccion',
       'Tipo de suelo_fachada', 'cocina_equipada', 'garaje', 'm2_utiles',
       'vista_zona', 'piscina', 'trastero', 'gastos_comunidad', 'Zonas_verdes',
       'terrazas', 'acceso_minusvalido', 'puerta_seguridad', 'conserje',
       'num_plantas', 'num_pisos', 'alarma', 'amueblado'],
      dtype='object')

In [103]:
redpiso_detalles2["referencia"]

0       REF RP2842024125161
1        REF RP782024130863
2        REF RP442024126007
3        REF RP442024126427
4       REF RP2842024130186
               ...         
2090     REF RP962024124554
2091    REF RP1922024124579
2095     REF RP272023121667
2096    REF RP2532022104355
2097    REF RP2332024131076
Name: referencia, Length: 1468, dtype: object

In [104]:
# Eliminar "REP" del campo "REF" 
redpiso_detalles2['referencia'] = redpiso_detalles2['referencia'].str.replace('REF', '').str.strip()

In [105]:
redpiso_detalles2['referencia']

0       RP2842024125161
1        RP782024130863
2        RP442024126007
3        RP442024126427
4       RP2842024130186
             ...       
2090     RP962024124554
2091    RP1922024124579
2095     RP272023121667
2096    RP2532022104355
2097    RP2332024131076
Name: referencia, Length: 1468, dtype: object

In [106]:
redpiso_normal2["referencia"]

0       RP2842024125161
1        RP782024130863
2        RP442024126007
3        RP442024126427
4       RP2842024130186
             ...       
2090     RP962024124554
2091    RP1922024124579
2095     RP272023121667
2096    RP2532022104355
2097    RP2332024131076
Name: referencia, Length: 1468, dtype: object

In [107]:
df_merge= pd.merge(redpiso_detalles2,redpiso_normal2,how="inner",on=["referencia"],indicator=True)

In [108]:
df_merge.shape

(1470, 48)

In [109]:
df_merge.duplicated().sum()

0

# silvia me llego aqui dia 11

In [110]:
df_merge.isna().sum()

referencia                     0
cantidad_visitas               0
fecha_publicacion              0
espacio_metros_cuadrados       2
banos                          7
campo10                      508
campo11                       12
campo12                     1445
dormitorios                   29
cantidad_armarios            884
consumo_energia             1210
emisiones_CO2               1209
cal_energetica               476
estado                       218
antiguedad                  1139
ascensor                       0
aire_acondicionado             0
orientacion                  715
agua_caliente                778
calefaccion                  890
Tipo de suelo_fachada        956
cocina_equipada                0
garaje                      1163
m2_utiles                      2
vista_zona                   342
piscina                        0
trastero                       0
gastos_comunidad            1439
Zonas_verdes                   0
terrazas                    1426
acceso_min

In [111]:
df_merge.columns

Index(['referencia', 'cantidad_visitas', 'fecha_publicacion',
       'espacio_metros_cuadrados', 'banos', 'campo10', 'campo11', 'campo12',
       'dormitorios', 'cantidad_armarios', 'consumo_energia', 'emisiones_CO2',
       'cal_energetica', 'estado', 'antiguedad', 'ascensor',
       'aire_acondicionado', 'orientacion', 'agua_caliente', 'calefaccion',
       'Tipo de suelo_fachada', 'cocina_equipada', 'garaje', 'm2_utiles',
       'vista_zona', 'piscina', 'trastero', 'gastos_comunidad', 'Zonas_verdes',
       'terrazas', 'acceso_minusvalido', 'puerta_seguridad', 'conserje',
       'num_plantas', 'num_pisos', 'alarma', 'amueblado', 'precio',
       'url_imagen', 'reservado', 'm2_constr', 'cantidad_dormitorios', 'url',
       'titulo', 'contacto', 'officelink_url', 'nombre_oficina', '_merge'],
      dtype='object')

In [112]:
#coleliminar=["imagen4","cantidad_dormitorios","número5",'officelink_url']

In [113]:
pd.set_option('display.max_colwidth', None)

In [114]:
df_merge= df_merge.rename(columns={'campo10': 'titular_anuncio','campo11':'descripcion_extendida','espacio_metros_cuadrados':'m2constr'})

In [115]:
df_merge["campo12"].value_counts(dropna=False)

campo12
NaN           1445
Adosado         19
Pareado          4
Individual       2
Name: count, dtype: int64

In [116]:
df_merge['tipologia'] = df_merge['titulo'].str.split().str[0]

In [117]:
df_merge['tipologia'].value_counts(dropna=False)

tipologia
Piso           1335
Ático            38
Apartamento      28
Estudio          22
Casa             19
Dúplex           11
Loft              7
Chalet            7
Venta             3
Name: count, dtype: int64

In [118]:


# Definir una función para concatenar solo si no hay nulos
def concatenar_si_no_nulo(row):
    if pd.notnull(row['tipologia']) and pd.notnull(row['campo12']):
        return row['tipologia'] + ' ' + row['campo12']
    else:
        return row['tipologia']  # O cualquier otro valor que desees en caso de nulos

# Aplicar la función a cada fila del DataFrame
df_merge['tipologia'] = df_merge.apply(concatenar_si_no_nulo, axis=1)




In [119]:
df_merge['tipologia'].value_counts(dropna=False)

tipologia
Piso               1335
Ático                38
Apartamento          28
Estudio              22
Casa Adosado         15
Dúplex               11
Loft                  7
Chalet Adosado        4
Venta                 3
Casa Individual       2
Casa Pareado          2
Chalet Pareado        2
Chalet                1
Name: count, dtype: int64

In [120]:
coleliminar=["imagen4","cantidad_dormitorios","número5",'officelink_url','campo12', 'url_imagen']

In [121]:
redpiso_unido= df_merge

In [122]:
# Creacion de nuevas columnas donde metere los valores de de distrito, barrios, codigos postal y el codigo por distrito y barrio.

nuevas_columnas= ["barrios","cod_barrio","cod_barrio_2","distrito","cod_distrito","codigo_Postal"]

redpiso_unido[nuevas_columnas]=np.nan

In [123]:
redpiso_unido2=redpiso_unido

#### **_Agregacion de barrios y distritos con sus codigos respectivos/ Redpiso._**

In [124]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower()  # Convertir a minúsculas
    return s

In [125]:
# Aqui hacemos la normalizacion de las columnas para despues hacer una mejor filtracion de nombres de distritos y barrios. 



def clasificar_distrito_y_barrio_por_columnas(df_sin_duplicados, dbcp_ordenado):
    # Crear las listas de barrios y distritos desde el dataframe de referencia
    distritos = dbcp_ordenado["nombre_distrito"].apply(lambda x: normalize_string(x)).tolist()
    barrios = dbcp_ordenado["nombre_barrio"].apply(lambda x: normalize_string(x)).tolist()

    # Recorrer las filas del dataframe
    for idx, fila in df_sin_duplicados.iterrows():
        # Recorrer las tres columnas en orden de prioridad: descripcion, descripcion_extendida, url_website
        for columna in ['titulo', 'nombre_oficina']:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)
                
                # Verificamos si el valor normalizado contiene el nombre de un distrito o barrio
                for distrito, barrio in zip(distritos, barrios):
                    if distrito in valor_normalizado and pd.isna(fila["distrito"]):
                        df_sin_duplicados.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
                    if barrio in valor_normalizado and pd.isna(fila["barrios"]):
                        df_sin_duplicados.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"

    return df_sin_duplicados

# Llamar a la función pasándole el dataframe y el dataframe de referencia
df_sin_duplicados = clasificar_distrito_y_barrio_por_columnas(redpiso_unido2, dbcp_ordenado)

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1962636392.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'villaverde' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sin_duplicados.at[idx, "distrito"] = distrito  # Asignar el nombre del distrito a la columna "distrito"
C:\Users\extas\AppData\Local\Temp\ipykernel_20344\1962636392.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'san cristobal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_sin_duplicados.at[idx, "barrios"] = barrio  # Asignar el nombre del barrio a la columna "barrio"


In [ ]:
redpiso_unido2.head(5)

In [127]:
def rellenar_distritos(redpiso_unido2, dbcp_ordenado):
    # Crear un diccionario de barrios a distritos normalizado
    barrio_a_distrito = {normalize_string(barrio): distrito for barrio, distrito in zip(dbcp_ordenado["nombre_barrio"], dbcp_ordenado["nombre_distrito"])}

    for idx, fila in redpiso_unido2.iterrows():
        # Verificamos si el valor de distrito está vacío y si hay un valor en barrios
        if pd.isna(fila["distrito"]) and not pd.isna(fila["barrios"]):
            barrio_normalizado = normalize_string(fila["barrios"])  # Normalizar el valor del barrio
            distrito = barrio_a_distrito.get(barrio_normalizado)  # Buscar el distrito usando el barrio normalizado
            if distrito:
                redpiso_unido2.at[idx, "distrito"] = distrito  # Asignar el distrito correspondiente

# Llamar a la función pasándole el dataframe y el dataframe de referencia
rellenar_distritos(redpiso_unido2=redpiso_unido2, dbcp_ordenado=dbcp_ordenado)


In [128]:

diccionario_ditrito_barrio= {
    "Centro": ["Malasaña", "La Latina", "Chueca","malasana"],
    "Moratalaz": ["La Elipa"],
    "Moncloa-Aravaca": ["Puerta de Hierro", "Príncipe Pío"],
    "Fuencarral-El Pardo": ["Buena Vista", "Ventilla", "Fuencarral"],
    "Chamartín": ["Bernabéu"],
    "Ciudad Lineal": ["Arturo Soria"],
    "San Blas-Canillejas": ["Las Mercedes","las mercedes"],
    "Villaverde": ["Villaverde Alto", "Villaverde Bajo"],
    "Carabanchel": ["Pau Carabanchel"],
}


# Función para asignar barrios y distritos recorriendo solo el diccionario
def asignar_barrios_y_distritos(df, diccionario, columnas_busqueda):
    # Normalizar los barrios en el diccionario
    diccionario_normalizado = {
        normalize_string(barrio): distrito for distrito, barrios in diccionario.items() for barrio in barrios
    }
    
    # Asegurar que las columnas "barrios" y "distritos" existan en el DataFrame
    3#if "barrios" not in df.columns:
     #   df["barrios"] = pd.NA
    #if "distrito" not in df.columns:
     #   df["distrito"] = pd.NA

    # Iterar sobre el diccionario de barrios y distritos
    for barrio_normalizado, distrito in diccionario_normalizado.items():
        # Buscar coincidencias en las columnas especificadas
        for columna in columnas_busqueda:
            # Generar una máscara para las coincidencias
            mask = df[columna].str.contains(barrio_normalizado, na=False, case=False)

            # Asignar valores donde haya coincidencias y columnas vacías
            df.loc[mask & df['barrios'].isna(), 'barrios'] = barrio_normalizado
            df.loc[mask & df['distrito'].isna(), 'distrito'] = distrito

    return df

# Llamar a la función
columnas_busqueda = ["barrios", "distrito"]  # Ajusta estas columnas según tu DataFrame
redpiso_unido2 = asignar_barrios_y_distritos(redpiso_unido2, diccionario_ditrito_barrio, columnas_busqueda)



In [129]:
def asignar_codigos(df_sin_duplicados, dbcp_ordenado):
    """
    Asigna los valores de cod_distrito, cod_barrio, cod_barrio_2 y codigo_postal al DataFrame original
    basándose en la correspondencia con el DataFrame de referencia.
    """
    # Normalizar los nombres en el DataFrame de referencia
    dbcp_ordenado['nombre_distrito_normalizado'] = dbcp_ordenado['nombre_distrito'].apply(normalize_string)
    dbcp_ordenado['nombre_barrio_normalizado'] = dbcp_ordenado['nombre_barrio'].apply(normalize_string)

    # Normalizar las columnas 'distrito' y 'barrios' en el DataFrame original
    df_sin_duplicados['distrito_normalizado'] = df_sin_duplicados['distrito'].apply(normalize_string)
    df_sin_duplicados['barrios_normalizado'] = df_sin_duplicados['barrios'].apply(normalize_string)

    # Realizar asignaciones basadas en coincidencias
    for idx, fila in df_sin_duplicados.iterrows():
        # Verificar coincidencia por distrito
        if pd.notna(fila['distrito_normalizado']):
            coincidencias_distrito = dbcp_ordenado[
                dbcp_ordenado['nombre_distrito_normalizado'] == fila['distrito_normalizado']
            ]
            if not coincidencias_distrito.empty:
                df_sin_duplicados.at[idx, 'cod_distrito'] = coincidencias_distrito['cod_distrito'].iloc[0]

        # Verificar coincidencia por barrio
        if pd.notna(fila['barrios_normalizado']):
            coincidencias_barrio = dbcp_ordenado[
                dbcp_ordenado['nombre_barrio_normalizado'] == fila['barrios_normalizado']
            ]
            if not coincidencias_barrio.empty:
                df_sin_duplicados.at[idx, 'cod_barrio'] = coincidencias_barrio['cod_barrio'].iloc[0]
                df_sin_duplicados.at[idx, 'cod_barrio_2'] = coincidencias_barrio['cod_barrio_2'].iloc[0]
                df_sin_duplicados.at[idx, 'codigo_Postal'] = coincidencias_barrio['codigo_Postal'].iloc[0]

    # Eliminar columnas normalizadas después de asignaciones
    df_sin_duplicados.drop(columns=['distrito_normalizado', 'barrios_normalizado'], inplace=True)
    dbcp_ordenado.drop(columns=['nombre_distrito_normalizado', 'nombre_barrio_normalizado'], inplace=True)

    return df_sin_duplicados

# Llamar a la función con los DataFrames correspondientes
redpiso_unido2 = asignar_codigos(redpiso_unido2, dbcp_ordenado)

In [130]:
redpiso_unido2.columns

Index(['referencia', 'cantidad_visitas', 'fecha_publicacion', 'm2constr',
       'banos', 'titular_anuncio', 'descripcion_extendida', 'campo12',
       'dormitorios', 'cantidad_armarios', 'consumo_energia', 'emisiones_CO2',
       'cal_energetica', 'estado', 'antiguedad', 'ascensor',
       'aire_acondicionado', 'orientacion', 'agua_caliente', 'calefaccion',
       'Tipo de suelo_fachada', 'cocina_equipada', 'garaje', 'm2_utiles',
       'vista_zona', 'piscina', 'trastero', 'gastos_comunidad', 'Zonas_verdes',
       'terrazas', 'acceso_minusvalido', 'puerta_seguridad', 'conserje',
       'num_plantas', 'num_pisos', 'alarma', 'amueblado', 'precio',
       'url_imagen', 'reservado', 'm2_constr', 'cantidad_dormitorios', 'url',
       'titulo', 'contacto', 'officelink_url', 'nombre_oficina', '_merge',
       'tipologia', 'barrios', 'cod_barrio', 'cod_barrio_2', 'distrito',
       'cod_distrito', 'codigo_Postal'],
      dtype='object')

------------------------------------------

### **_Aqui se termina la union de redpiso y empieza la union de redpiso y pisosmadrid_**

In [131]:
# eliminacion de columnas 

redpiso_columnas_eliminar=["url_imagen","nombre_oficina","contacto","officelink_url"]

redpiso_unido2 = redpiso_unido2.drop(columns=redpiso_columnas_eliminar)

In [132]:
redpiso_unido2.shape

(1470, 51)

In [133]:
redpiso_unido2["web"]="redpiso"
redpiso_unido2["fecha_descarga"]="28/10/2024"

In [135]:
redpiso_unido2["Tipo de suelo_fachada"].value_counts()

Tipo de suelo_fachada
Tipo de suelo Gres                  129
Tipo de suelo Parquet               112
Tipo de fachada Ladrillo             70
Tipo de suelo Tarima normal          61
Tipo de suelo Tarima Flotante        53
Tipo de suelo Baldosa                16
Tipo de suelo Sintasol               15
Tipo de suelo Terrazo                14
Tipo de suelo Otros                  12
Tipo de fachada Mono Capa            11
Tipo de fachada Cemento               8
Tipo de suelo Porcelanico             6
Tipo de suelo Mosaico Hidraulico      3
Tipo de fachada Piedra                2
Tipo de fachada Otros materiales      1
Tipo de suelo Marmol                  1
Name: count, dtype: int64

In [136]:
# Aqui vamos a filtrar la informacion de tipo_fachada y la dividire en dos columnas donde se veran los valores de fachada y suelo separados
redpiso_unido2["tipo_fachada"]= np.nan

def asignar_fachada(df):
    # Recorrer cada fila de la columna Tipo de suelo/fachada
    for idx, fila in df.iterrows():
        tipo_suelo_fachada = str(fila['Tipo de suelo_fachada'])  # Asegurarse de que sea un string
        # Convertir el texto a minúsculas para la búsqueda sin importar mayúsculas
        if 'fachada' in tipo_suelo_fachada.lower():
            # Si encuentra la palabra 'fachada', asignar todo el texto a la columna tipo_fachada
            df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada
            # Reemplazar el valor original en Tipo de suelo/fachada por NaN
            df.at[idx, 'Tipo de suelo_fachada'] = np.nan
    return df

# Llamar a la función pasándole tu dataframe
redpiso_unido2 = asignar_fachada(redpiso_unido2)

C:\Users\extas\AppData\Local\Temp\ipykernel_20344\3383542287.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tipo de fachada Ladrillo' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada


In [137]:
redpiso_unido2["Tipo de suelo_fachada"].value_counts()

Tipo de suelo_fachada
Tipo de suelo Gres                  129
Tipo de suelo Parquet               112
Tipo de suelo Tarima normal          61
Tipo de suelo Tarima Flotante        53
Tipo de suelo Baldosa                16
Tipo de suelo Sintasol               15
Tipo de suelo Terrazo                14
Tipo de suelo Otros                  12
Tipo de suelo Porcelanico             6
Tipo de suelo Mosaico Hidraulico      3
Tipo de suelo Marmol                  1
Name: count, dtype: int64

In [138]:
redpiso_unido2["tipo_fachada"].value_counts()

tipo_fachada
Tipo de fachada Ladrillo            70
Tipo de fachada Mono Capa           11
Tipo de fachada Cemento              8
Tipo de fachada Piedra               2
Tipo de fachada Otros materiales     1
Name: count, dtype: int64

In [158]:
# renombramiento de columna 
redpiso_unido2= redpiso_unido2.rename(columns={"Tipo de suelo_fachada":"tipo_suelo"})
redpiso_unido2= redpiso_unido2.rename(columns={"barrios":"barrio"})
redpiso_unido2= redpiso_unido2.rename(columns={"Zonas_verdes":"zonas_verdes"})
redpiso_unido2= redpiso_unido2.rename(columns={"codigo_Postal":"codigo_postal"})

In [159]:
redpiso_unido2.columns

Index(['referencia', 'cantidad_visitas', 'fecha_publicacion', 'banos',
       'titular_anuncio', 'descripcion_extendida', 'campo12', 'dormitorios',
       'cantidad_armarios', 'consumo_energia', 'emisiones_CO2',
       'cal_energetica', 'estado', 'antiguedad', 'ascensor',
       'aire_acondicionado', 'orientacion', 'agua_caliente', 'calefaccion',
       'tipo_suelo', 'cocina_equipada', 'garaje', 'm2_utiles', 'vista_zona',
       'piscina', 'trastero', 'gastos_comunidad', 'zonas_verdes', 'terrazas',
       'acceso_minusvalido', 'puerta_seguridad', 'conserje', 'num_plantas',
       'num_pisos', 'alarma', 'amueblado', 'precio', 'url_imagen', 'reservado',
       'm2_constr', 'cantidad_dormitorios', 'url', 'titulo', 'nombre_oficina',
       'tipologia', 'barrio', 'cod_barrio', 'cod_barrio_2', 'distrito',
       'cod_distrito', 'codigo_postal'],
      dtype='object')

In [160]:
columnas_a_eliminar = ["_merge","officelink_url","contacto","m2constr"]

redpiso_unido2= redpiso_unido2.drop(columns=columnas_a_eliminar)

KeyError: "['_merge', 'officelink_url', 'contacto', 'm2constr'] not found in axis"

In [161]:
redpiso_unido2.to_csv("red_pisoparaunir.csv",index=False)

In [150]:
redpiso_unido2[['url']].value_counts(dropna=False)

url                                                                                                                             
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-pinzon-vista-alegre-carabanchel-madrid-madrid-RP302023121517                 4
https://www.redpiso.es/inmueble/apartamento-en-venta-en-calle-alberto-aguilera-gaztambide-chamberi-madrid-madrid-RP1782024122328    1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-portalegre-opanel-carabanchel-madrid-madrid-RP2412024124127                  1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-principe-de-vergara-hispanoamerica-chamartin-madrid-madrid-RP902023109627    1
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-principe-de-vergara-hispanoamerica-chamartin-madrid-madrid-RP902022102747    1
                                                                                                                                   ..
https://www.redpiso.es/inmueble/piso-en-venta-en-calle-colomer-guin

In [143]:
redpiso_unido2.to_csv("red_pisoparaunir.csv",index=False)

### **_Desde aqui empiezan los ajustes de compañera Silvia_**

In [333]:
# dfi= pd.read_feather('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/df_Indomio22122024.ftr')
# dfr_pm= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/redpiso_pisosmadrid_unidos.csv')

In [334]:
# dfi.columns

In [335]:

# # Eliminar el símbolo € y la cadena /mes
# dfi['€_comunidad_mes'] = dfi['gastos_comunidad'].replace({'€ ': '', '/mes': ''}, regex=True)
# dfi['acceso_minusvalido']=dfi['acceso_minusv'].str.replace('No','0').str.replace('Sí','1')
# #elimino la columna
# dfi=dfi.drop("acceso_minusv",axis=1)
# dfi=dfi.drop("gastos_comunidad",axis=1)

#en la columna antiguedad de redpiso_pisosmadrid2 quitar el texto año Ano de construccion del valor
redpiso_pisosmadrid2['antiguedad']=redpiso_pisosmadrid2['antiguedad'].str.replace('Ano de construccion','')
redpiso_pisosmadrid2['antiguedad']=redpiso_pisosmadrid2['antiguedad'].str.replace('Antiguedad','')
# redpiso_pisosmadrid2["plantas"]=redpiso_pisosmadrid2['plantas'].str.replace('Plantas','')
# Crear la nueva columna 'exterior' y asignar el valor 1 cuando 'vista zona' sea 'Exterior'
# redpiso_pisosmadrid2=redpiso_pisosmadrid2.drop("plantas",axis=1)

redpiso_pisosmadrid2['exterior'] = redpiso_pisosmadrid2['vista zona'].apply(lambda x: 1 if x == 'Exterior' else np.nan)
# redpiso_pisosmadrid2=redpiso_pisosmadrid2.drop("vista zona",axis=1)


In [336]:
redpiso_pisosmadrid2["antiguedad"].value_counts()

antiguedad
Mas de 50 anos         131
Entre 35 y 50 Anos      29
1960                    15
1965                    15
1930                    13
1940                     9
1964                     7
1972                     7
Mas de 25 Anos           6
1970                     6
1925                     5
1962                     5
1966                     5
1950                     4
1900                     4
2000                     4
1959                     4
1920                     3
Menos de 5 Anos          3
1968                     3
1961                     3
1969                     3
1955                     3
1973                     2
1967                     2
1923                     2
1994                     2
 Entre 20 y 25 Anos      2
1979                     2
1910                     2
1929                     2
1957                     2
1986                     2
1958                     1
45                       1
1928                     1
1987             

In [337]:
redpiso_pisosmadrid2.head(1)

,referencia_web,cantidad_visitas,espacio_metros_cuadrados,cant_dormitorios,cant_banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,condicino-estado,antiguedad,ascensor,orientacion,sistema_agua,sistema_calefaccion,tipo_suelo,cocina_equipada,garajes,m_cuadrados_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,calefaccion,chimenea,lavadero,videoportero,titulo,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2,tipo_fachada,exterior
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN


In [338]:
# # Reemplazar "Sí" por 1 y "No" por 0 
# dfi['ascensor'] = dfi['ascensor'].replace({'Sí': 1, 'No': 0})
# dfi['balcon'] = dfi['balcon'].replace({'Sí': 1, 'No': 0})

In [339]:
# Diccionario con los nuevos nombres de las columnas
#voy a suponer que la superficie es superficie construida
nuevos_nombres ={'titulo': 'title',
'espacio_metros_cuadrados': 'm2_constr',
'm_cuadrados_utiles':'m2_utiles',
'cant_dormitorios': 'dormitorios',
'cant_banos': 'banos' ,
'condicino-estado':'estado',
'sistema_calefaccion':'calefaccion',
'antiguedad' :'ano_construccion'} 
redpiso_pisosmadrid2 = redpiso_pisosmadrid2.rename(columns=nuevos_nombres)

In [340]:
redpiso_pisosmadrid2.head(5)

,referencia_web,cantidad_visitas,m2_constr,dormitorios,banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,estado,ano_construccion,ascensor,orientacion,sistema_agua,calefaccion,tipo_suelo,cocina_equipada,garajes,m2_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,calefaccion,chimenea,lavadero,videoportero,title,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2,tipo_fachada,exterior
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
1,NaN,NaN,24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39473...,Garaje 0 dormitorios 0 banos 0 garajes Cerrado...,0.0,0.0,PM3945003,60.0,0.0,0.0,0.0,0.0,Garaje en venta en Pau Carabanchel Madrid,15500.0,2024-07-11 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
2,NaN,NaN,340,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/40017...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,1.0,0.0,PM4000060,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,3000000.0,2024-08-29 00:00:00,0.0,0.0,1.0,1.0,1.0,1,NaN,NaN
3,NaN,NaN,399,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,local,https://www.pisosmadrid.com.es/propiedad/40407...,Amplio local o nave de 399 m2 distribuido en d...,0.0,0.0,PM4040343,30.0,0.0,0.0,0.0,0.0,Local en venta en Fuencarral Madrid,409100.0,2024-09-26 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
4,NaN,NaN,442,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/39862...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,0.0,0.0,PM3984633,50.0,1.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,8000000.0,2024-08-09 00:00:00,0.0,0.0,0.0,1.0,1.0,1,NaN,NaN


In [341]:
# redpiso_pisosmadrid2= redpiso_pisosmadrid2.rename(columns={"antiguedad ":"ano_antiguedad"})

In [342]:
# redpiso_pisosmadrid2["antieguedad"].value_counts()

In [343]:
# nuevos_nombres ={'sistema_alarma': 'alarma',
# 'cocina_1': 'cocina',
# 'orientacion_sur,_este':'orientacion_sur_este',
# 'orientacion_norte,_sur':'orientacion_norte_sur'
#  } 
# dfi = dfi.rename(columns=nuevos_nombres)

In [346]:
redpiso_pisosmadrid2.shape

(3311, 56)

In [347]:
# dfi.columns

In [348]:
# dfi=dfi.drop("orientacion", axis=1)

In [349]:
# dfi.shape

In [350]:
# # Usar melt para transformar el DataFrame
# df_melted = pd.melt(dfi, id_vars=['url'], value_vars=['orientacion_este', 'orientacion_norte',
#        'orientacion_norte_sur', 'orientacion_oeste', 'orientacion_sur',
#        'orientacion_sur_este'],var_name='orientacion', value_name='valor')

# # Filtrar las filas donde el valor es 1
# df_melted = df_melted[df_melted['valor'] == 1]

# # Eliminar la columna 'valor'
# df_melted = df_melted.drop(columns=['valor'])

# # Extraer la orientación real
# df_melted['orientacion'] = df_melted['orientacion'].str.replace('orientacion_', '')

In [351]:
# dfi.shape

In [352]:
# #uno al dataframe dfi
# dfi=dfi.merge(df_melted,on="url",how="left")

In [353]:
# dfi.loc[dfi['orientacion'].notnull(), ['orientacion', 'orientacion_este', 'orientacion_norte', 'orientacion_norte_sur', 'orientacion_oeste', 'orientacion_sur', 'orientacion_sur_este']]


In [354]:
# # Eliminar las columnas originales
# dfi = dfi.drop(columns=['orientacion_este', 'orientacion_norte', 'orientacion_norte_sur', 'orientacion_oeste', 'orientacion_sur', 'orientacion_sur_este'])



In [355]:
# Asegurarse de que todos los nombres de las columnas sean cadenas de texto 
redpiso_pisosmadrid2.columns = redpiso_pisosmadrid2.columns.astype(str) 
# Eliminar espacios en blanco al principio o al final de los nombres de las columnas
redpiso_pisosmadrid2.columns = redpiso_pisosmadrid2.columns.str.strip()

In [356]:
# dfi.to_csv("dUnionSilvia23122024.csv")

In [357]:
# dfr_pm.to_csv("dUnionGilberto23122024.csv")

In [358]:
# def asignar_zona_distrito_barrio(row):
#     if row['zona'] == 'Sanchinarro':
#         row['distrito'] = 'Hortaleza'
#         row['barrio'] = 'Valdefuentes'
#     if row['zona'] == 'San Andrés':
#         row['distrito'] = 'Villaverde'
#         row['barrio'] = 'VillaverdeAlto,CascoHistóricodeVillaverde'
#     if row['zona'] == 'Virgen del Cortijo':
#         row['distrito'] = 'Hortaleza'
#         row['barrio'] = 'Valdefuentes'
#     if row['zona'] == 'Montecarmelo':
#         row['distrito'] = 'Fuencarral-El Pardo'
#         row['barrio'] = 'Mirasierra'

#     return row


In [359]:
redpiso_pisosmadrid2

,referencia_web,cantidad_visitas,m2_constr,dormitorios,banos,cantidad_armarios,consumo_energia,emisiones_CO2,cal_energetica,estado,ano_construccion,ascensor,orientacion,sistema_agua,calefaccion,tipo_suelo,cocina_equipada,garajes,m2_utiles,vista zona,Comunidad/personas edificio,Zonas verdes,terrazas,acceso_minusvalido,puerta_seguridad,conserje,plantas,N_piso,reservado/sin_reserva,url_piso/id,barrios,cod_barrio,distrito,cod_distrito,codigo_Postal,tipologia,url_website,descripcion_extendida,garaje,jardin,referencia,n.visitas,chimenea,lavadero,videoportero,title,precio,fecha_publicacion,aire_acondicionado,piscina,trastero,alarma,amueblado,cod_barrio_2,tipo_fachada,exterior
0,NaN,NaN,12,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39778...,Servicheck presenta las mejores plazas de gara...,0.0,0.0,PM3974960,70.0,0.0,0.0,0.0,Garaje en venta en Tetuan Madrid,25000.0,2024-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
1,NaN,NaN,24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,garaje,https://www.pisosmadrid.com.es/propiedad/39473...,Garaje 0 dormitorios 0 banos 0 garajes Cerrado...,0.0,0.0,PM3945003,60.0,0.0,0.0,0.0,Garaje en venta en Pau Carabanchel Madrid,15500.0,2024-07-11 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
2,NaN,NaN,340,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/40017...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,1.0,0.0,PM4000060,50.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,3000000.0,2024-08-29 00:00:00,0.0,0.0,1.0,1.0,1.0,1,NaN,NaN
3,NaN,NaN,399,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,goya,42,salamanca,4,28001,local,https://www.pisosmadrid.com.es/propiedad/40407...,Amplio local o nave de 399 m2 distribuido en d...,0.0,0.0,PM4040343,30.0,0.0,0.0,0.0,Local en venta en Fuencarral Madrid,409100.0,2024-09-26 00:00:00,0.0,0.0,0.0,0.0,0.0,2,NaN,NaN
4,NaN,NaN,442,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recoletos,41,salamanca,4,28001,negocio,https://www.pisosmadrid.com.es/propiedad/39862...,TGG INMOBILIARIA vende en exclusiva NEGOCIO DE...,0.0,0.0,PM3984633,50.0,0.0,0.0,0.0,Negocio en venta en Recoletos Madrid,8000000.0,2024-08-09 00:00:00,0.0,0.0,0.0,1.0,1.0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,NaN,NaN,36,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmuebleduplexenventaencallej...,nan,nan,barajas,21.0,nan,duplex,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dúplex en venta en CALLE JUPITER Casco Históri...,190000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3307,NaN,NaN,57,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencallenue...,nan,nan,barajas,21.0,nan,piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Piso en venta en CALLE NUESTRA SEÑORA DE ARACE...,205000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3308,NaN,NaN,63,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,httpswwwredpisoesinmueblepisoenventaencalleemp...,nan,nan,barajas,21.0,nan,piso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Piso en venta en CALLE EMPEDRADA Casco Históri...,232000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3309,NaN,NaN,68,nan,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,httpswwwredpisoesinmueblepisoenventaencalleemp...,nan,nan,ba

In [362]:
redpiso_pisosmadrid2.to_csv("redpiso_pisosmadrid.csv",index= False)